# PyMixer demo

This notebook contains code snippets showing how `pymixer` package can be used.

### Setup

Let us import required modules, classes, and functions.

In [ ]:
import os

import IPython.display
import numpy as np
import pretty_midi
from pymixer.core import FluidsynthMidiInput, SinethesizerMidiInput, Project
from pymixer.midi import merge_midi_objects, split_midi_file_by_instruments
from sinethesizer.synth.core import Event
from sinethesizer.effects.reverb import apply_room_reverb
from sinethesizer.effects.stereo import apply_stereo_to_mono_conversion
from sinethesizer.io.events_to_wav import write_timeline_to_wav

Then, suppose that three directories are defined: one with input MIDI files, one for saving intermediate data, and one for saving final outputs.

In [ ]:
input_dir = "..."  # Insert your value.
intermediate_dir = "..."  # Insert your value.
output_dir = "..."  # Insert your value.

### Case 1. MIDI merging

Given directory with MIDI files, combine all of them in sequence into a single MIDI file. Precedence of the files should be defined by their names.

In [ ]:
midi_objects = []
for file_name in sorted(os.listdir(input_dir)):
    if not file_name.endswith('.mid'):
        continue
    midi_objects.append(pretty_midi.PrettyMIDI(f'{input_dir}/{file_name}'))

In [ ]:
merged_midi_object = merge_midi_objects(
    midi_objects,
    opening_silence_in_sec=1.0,
    trailing_silence_in_sec=1.0,
    # Below value assumes that there are exactly 3 files in the directory and no silence between them is needed.
    caesuras_in_sec=[0, 0],
    # Below value assumes that there are 3 tracks named 0, 1, and 2, and program 2 should be used for all of them.
    instrument_name_to_program={0: 2, 1: 2, 2: 2, 3: 2}
)
track_path = f'{intermediate_dir}/track.mid'
merged_midi_object.write(track_path)

### Case 2. Saving each MIDI track to a separate MIDI file

In [ ]:
split_midi_file_by_instruments(track_path, intermediate_dir)

### Case 3. Creating PyMixer project

In [ ]:
soundfont_path = '...'  # Insert your path.
sinethesizer_presets_path = '...'  # Insert your path.

In [ ]:
inputs = [
    FluidsynthMidiInput(
        f'{intermediate_dir}/track_1.mid',
        soundfont_path,
        fluidsynth_chorus=True,
        fluidsynth_reverb=False,
    ),
    FluidsynthMidiInput(
        f'{intermediate_dir}/track_2.mid',
        soundfont_path,
        fluidsynth_chorus=True,
        fluidsynth_reverb=False,
    ),
    SinethesizerMidiInput(
        f'{intermediate_dir}/track_3.mid',
        sinethesizer_presets_path,
        track_name_to_instrument={'3': 'bowed_string'}
    )
]

In [ ]:
project = Project(inputs, frame_rate=48000)

### Case 4. Applying effects to tracks

Actually, every function that takes `numpy` array as input and returns `numpy` array as output can be used for applying effects. In this tutorial, `sinethesizer` effects are chosen as an example.

A `sinethesizer` event is needed for all `sinethesizer` effects. For some effects, this event affects the output, but this is not the case for reverb and stereo-to-mono conversion. So let us create a placeholder. 

In [ ]:
dummy_event = Event(
    instrument='does_not_matter',
    start_time=0,
    duration=1,
    frequency=1,
    velocity=1,
    effects="",
    frame_rate=project.frame_rate
)

Now, apply the effects. Initial panning is erased with stereo-to-mono conversion and then room reverb creates new panning based on simulation of sound waves reflections.

In [ ]:
project.tracks[0] = apply_stereo_to_mono_conversion(project.tracks[0], dummy_event)
project.tracks[0] = apply_room_reverb(
    project.tracks[0],
    dummy_event,
    **{
        "room_length": 65, "room_width": 25, "room_height": 15,
        "reflection_decay_factor": 0.7, "sound_speed": 343,
        "listener_x": 45, "listener_y": 12.5, "listener_z": 1.7,
        "listener_direction_x": 1, "listener_direction_y": 0,
        "sound_source_x": 57.124355652982146, "sound_source_y": 19.5, "sound_source_z": 4,
        "sound_source_direction_x": -0.8660254037844387, "sound_source_direction_y": -0.5, "sound_source_direction_z": 0,
        "angle": 1.5707963267948966, "n_reflections": 30
    }
)

project.tracks[1] = apply_stereo_to_mono_conversion(project.tracks[1], dummy_event)
project.tracks[1] = apply_room_reverb(
    project.tracks[1],
    dummy_event,
    **{
        "room_length": 65, "room_width": 25, "room_height": 15,
        "reflection_decay_factor": 0.7, "sound_speed": 343,
        "listener_x": 45, "listener_y": 12.5, "listener_z": 1.7,
        "listener_direction_x": 1, "listener_direction_y": 0,
        "sound_source_x": 57.124355652982146, "sound_source_y": 5.5, "sound_source_z": 4,
        "sound_source_direction_x": -0.8660254037844387, "sound_source_direction_y": 0.5, "sound_source_direction_z": 0,
        "angle": 1.5707963267948966, "n_reflections": 30
    }
)

project.tracks[2] = apply_stereo_to_mono_conversion(project.tracks[2], dummy_event)
project.tracks[2] = apply_room_reverb(
    project.tracks[2],
    dummy_event,
    **{
        "room_length": 65, "room_width": 25, "room_height": 15,
        "reflection_decay_factor": 0.7, "sound_speed": 343,
        "listener_x": 45, "listener_y": 12.5, "listener_z": 1.7,
        "listener_direction_x": 1, "listener_direction_y": 0,
        "sound_source_x": 59.0, "sound_source_y": 12.5, "sound_source_z": 4,
        "sound_source_direction_x": -1.0, "sound_source_direction_y": -0.0, "sound_source_direction_z": 0,
        "angle": 1.5707963267948966, "n_reflections": 30
    }
)

### Case 5. Rehearsal of the project

In [ ]:
# Below, track gains can be changed easily until mixed sound is good enough.
timeline = project.mix(gains=[1.0, 1.0, 2.0])
timeline /= np.max(np.abs(timeline))
IPython.display.Audio(timeline, rate=project.frame_rate)

### Case 6. Saving WAV output

In [ ]:
write_timeline_to_wav(f'{output_dir}/result.wav', timeline, project.frame_rate)